In [ ]:
import pandas as pd
import numpy as np
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from datetime import datetime, timedelta
import json
import csv
from typing import List, Dict, Any
import logging

class IndianVRPSolver:
    def __init__(self):
        # Configure logging
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)
        
        # Define vehicle types and their constraints
        self.vehicle_types = {
            'Cargo Auto': {
                'capacity_weight': 500,
                'capacity_volume': 5,
                'avg_speed_kmh': 30,
                'cost_per_km': 15
            },
            'Tata Ace': {
                'capacity_weight': 750,
                'capacity_volume': 8,
                'avg_speed_kmh': 35,
                'cost_per_km': 18
            },
            'Mahindra Pickup': {
                'capacity_weight': 1200,
                'capacity_volume': 12,
                'avg_speed_kmh': 40,
                'cost_per_km': 22
            },
            'Mini Truck': {
                'capacity_weight': 2500,
                'capacity_volume': 20,
                'avg_speed_kmh': 35,
                'cost_per_km': 25
            }
        }
        
        # City-specific traffic multipliers
        self.city_traffic = {
            'Mumbai': 1.8,
            'Delhi': 1.7,
            'Bangalore': 1.6,
            'Hyderabad': 1.5,
            'Chennai': 1.5,
            'Pune': 1.4,
            'Kolkata': 1.6
        }

    def load_delivery_data(self, csv_path: str) -> pd.DataFrame:
        """Load delivery data from CSV file"""
        try:
            df = pd.read_csv(csv_path)
            required_columns = [
                'transaction_id', 'pickup_city', 'delivery_city',
                'pickup_lat', 'pickup_lon', 'delivery_lat', 'delivery_lon',
                'weight_kg', 'volume_m3', 'earliest_delivery', 'latest_delivery'
            ]
            
            # Verify required columns exist
            missing_columns = [col for col in required_columns if col not in df.columns]
            if missing_columns:
                raise ValueError(f"Missing required columns: {missing_columns}")
            
            return df
        except Exception as e:
            self.logger.error(f"Error loading CSV file: {str(e)}")
            raise

    def prepare_delivery_requests(self, df: pd.DataFrame) -> List[Dict[str, Any]]:
        """Convert DataFrame rows to delivery request format"""
        delivery_requests = []
        
        for _, row in df.iterrows():
            # Create pickup location request
            pickup_request = {
                'id': f"{row['transaction_id']}_pickup",
                'city': row['pickup_city'],
                'lat': row['pickup_lat'],
                'lon': row['pickup_lon'],
                'weight': row['weight_kg'],
                'volume': row['volume_m3'],
                'type': 'pickup',
                'transaction_id': row['transaction_id'],
                'time_window': (0, 1440)  # Full day for pickup
            }
            
            # Create delivery location request
            delivery_request = {
                'id': f"{row['transaction_id']}_delivery",
                'city': row['delivery_city'],
                'lat': row['delivery_lat'],
                'lon': row['delivery_lon'],
                'weight': -row['weight_kg'],  # Negative weight to indicate delivery
                'volume': -row['volume_m3'],
                'type': 'delivery',
                'transaction_id': row['transaction_id'],
                'time_window': self._parse_time_window(
                    row['earliest_delivery'],
                    row['latest_delivery']
                )
            }
            
            delivery_requests.extend([pickup_request, delivery_request])
        
        return delivery_requests

    def _parse_time_window(self, earliest: str, latest: str) -> tuple:
        """Convert time strings to minutes since midnight"""
        try:
            earliest_time = pd.to_datetime(earliest).time()
            latest_time = pd.to_datetime(latest).time()
            
            earliest_minutes = earliest_time.hour * 60 + earliest_time.minute
            latest_minutes = latest_time.hour * 60 + latest_time.minute
            
            return (earliest_minutes, latest_minutes)
        except Exception as e:
            self.logger.warning(f"Error parsing time window: {str(e)}")
            return (0, 1440)  # Default to full day

    def generate_vehicle_fleet(self, delivery_requests: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """Generate a fleet of vehicles based on delivery requirements"""
        total_weight = sum(req['weight'] for req in delivery_requests if req['type'] == 'pickup')
        total_volume = sum(req['volume'] for req in delivery_requests if req['type'] == 'pickup')
        
        fleet = []
        vehicle_id = 1
        
        # Sort vehicle types by capacity
        sorted_vehicles = sorted(
            self.vehicle_types.items(),
            key=lambda x: x[1]['capacity_weight']
        )
        
        remaining_weight = total_weight
        while remaining_weight > 0:
            # Find suitable vehicle type
            chosen_vehicle = None
            for vehicle_type, specs in sorted_vehicles:
                if specs['capacity_weight'] >= remaining_weight:
                    chosen_vehicle = (vehicle_type, specs)
                    break
            
            if not chosen_vehicle:
                chosen_vehicle = sorted_vehicles[-1]  # Use largest vehicle
            
            fleet.append({
                'id': f"V{str(vehicle_id).zfill(3)}",
                'type': chosen_vehicle[0],
                'capacity_weight': chosen_vehicle[1]['capacity_weight'],
                'capacity_volume': chosen_vehicle[1]['capacity_volume']
            })
            
            remaining_weight -= chosen_vehicle[1]['capacity_weight']
            vehicle_id += 1
        
        return fleet

    def solve_vrp_from_csv(self, csv_path: str, output_path: str = None):
        """Solve VRP problem from CSV file and save results"""
        try:
            # Load and prepare data
            df = self.load_delivery_data(csv_path)
            delivery_requests = self.prepare_delivery_requests(df)
            available_vehicles = self.generate_vehicle_fleet(delivery_requests)
            
            # Solve VRP
            routes = self.solve_vrp(delivery_requests, available_vehicles)
            
            if routes:
                # Save results
                self.save_solution_to_csv(routes, output_path or 'optimized_routes.csv')
                return routes
            else:
                self.logger.warning("No solution found")
                return None
                
        except Exception as e:
            self.logger.error(f"Error solving VRP: {str(e)}")
            raise

    def solve_vrp(self, delivery_requests: List[Dict[str, Any]], 
                 available_vehicles: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """Core VRP solving logic"""
        # Create routing model
        num_locations = len(delivery_requests)
        num_vehicles = len(available_vehicles)
        depot = 0
        
        manager = pywrapcp.RoutingIndexManager(num_locations, num_vehicles, depot)
        routing = pywrapcp.RoutingModel(manager)

        # Distance callback
        def distance_callback(from_index, to_index):
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            
            from_loc = delivery_requests[from_node]
            to_loc = delivery_requests[to_node]
            
            return int(self._calculate_distance(
                from_loc['lat'], from_loc['lon'],
                to_loc['lat'], to_loc['lon']
            ) * 100)  # Convert to integer for OR-Tools

        transit_callback_index = routing.RegisterTransitCallback(distance_callback)
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        # Add capacity constraints
        def demand_callback(from_index):
            from_node = manager.IndexToNode(from_index)
            return int(delivery_requests[from_node]['weight'])

        demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
        
        for vehicle_idx, vehicle in enumerate(available_vehicles):
            routing.AddDimension(
                demand_callback_index,
                0,  # null capacity slack
                int(vehicle['capacity_weight']),  # vehicle maximum capacity
                True,  # start cumul to zero
                f'Capacity_{vehicle_idx}'
            )

        # Add time windows
        def time_callback(from_index, to_index):
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            
            from_loc = delivery_requests[from_node]
            to_loc = delivery_requests[to_node]
            
            dist = self._calculate_distance(
                from_loc['lat'], from_loc['lon'],
                to_loc['lat'], to_loc['lon']
            )
            
            # Convert distance to time (minutes)
            return int(dist * 2)  # Assuming 30 km/h average speed

        time_callback_index = routing.RegisterTransitCallback(time_callback)
        routing.AddDimension(
            time_callback_index,
            30,  # Allow waiting time
            1440,  # Maximum day length in minutes
            False,  # Don't force start cumul to zero
            'Time'
        )
        
        time_dimension = routing.GetDimensionOrDie('Time')
        
        # Add time windows constraints
        for location_idx, request in enumerate(delivery_requests):
            index = manager.NodeToIndex(location_idx)
            time_dimension.CumulVar(index).SetRange(
                request['time_window'][0],
                request['time_window'][1]
            )

        # Set first solution strategy
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
        search_parameters.local_search_metaheuristic = (
            routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
        search_parameters.time_limit.FromSeconds(30)

        # Solve
        solution = routing.SolveWithParameters(search_parameters)
        
        if solution:
            return self._extract_solution(manager, routing, solution, 
                                       delivery_requests, available_vehicles)
        return None

    def _calculate_distance(self, lat1: float, lon1: float, 
                          lat2: float, lon2: float) -> float:
        """Calculate distance between two points in kilometers"""
        R = 6371  # Earth's radius in kilometers
        
        lat1_rad = np.radians(lat1)
        lat2_rad = np.radians(lat2)
        delta_lat = np.radians(lat2 - lat1)
        delta_lon = np.radians(lon2 - lon1)
        
        a = (np.sin(delta_lat/2) * np.sin(delta_lat/2) +
             np.cos(lat1_rad) * np.cos(lat2_rad) *
             np.sin(delta_lon/2) * np.sin(delta_lon/2))
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        
        return R * c

    def _extract_solution(self, manager, routing, solution, 
                         delivery_requests, available_vehicles):
        """Extract the solution into a readable format"""
        routes = []
        time_dimension = routing.GetDimensionOrDie('Time')
        
        for vehicle_idx, vehicle in enumerate(available_vehicles):
            index = routing.Start(vehicle_idx)
            
            if routing.IsEnd(index):
                continue
                
            route = {
                'vehicle_id': vehicle['id'],
                'vehicle_type': vehicle['type'],
                'stops': [],
                'total_distance': 0,
                'total_load': 0
            }
            
            while not routing.IsEnd(index):
                node_index = manager.IndexToNode(index)
                next_index = solution.Value(routing.NextVar(index))
                
                time_var = time_dimension.CumulVar(index)
                load_dimension = routing.GetDimensionOrDie(f'Capacity_{vehicle_idx}')
                load_var = load_dimension.CumulVar(index)
                
                stop_info = delivery_requests[node_index].copy()
                stop_info.update({
                    'arrival_time': solution.Min(time_var),
                    'departure_time': solution.Max(time_var),
                    'current_load': solution.Min(load_var)
                })
                
                route['stops'].append(stop_info)
                
                if not routing.IsEnd(next_index):
                    next_node_index = manager.IndexToNode(next_index)
                    route['total_distance'] += self._calculate_distance(
                        delivery_requests[node_index]['lat'],
                        delivery_requests[node_index]['lon'],
                        delivery_requests[next_node_index]['lat'],
                        delivery_requests[next_node_index]['lon']
                    )
                
                index = next_index
            
            if route['stops']:
                routes.append(route)
        
        return routes

    def save_solution_to_csv(self, routes: List[Dict[str, Any]], output_path: str):
        """Save the solution to a CSV file"""
        rows = []
        for route in routes:
            for stop in route['stops']:
                rows.append({
                    'vehicle_id': route['vehicle_id'],
                    'vehicle_type': route['vehicle_type'],
                    'stop_type': stop['type'],
                    'transaction_id': stop['transaction_id'],
                    'city': stop['city'],
                    'latitude': stop['lat'],
                    'longitude': stop['lon'],
                    'arrival_time': stop['arrival_time'],
                    'departure_time': stop['departure_time'],
                    'current_load': stop['current_load']
                })
        
        df = pd.DataFrame(rows)
        df.to_csv(output_path, index=False)
        self.logger.info(f"Solution saved to {output_path}")

# Example usage
if __name__ == "__main__":
    solver = IndianVRPSolver()
    
    # Solve VRP from CSV
    routes = solver.solve_vrp_from_csv(
        csv_path='indian_delivery_transactions.csv',
        output_path='optimized_routes.csv'
    )
    
    if routes:
        print(f"Generated {len(routes)} routes")